# 🥈 Camada Silver: Refino e Padronização

Nesta etapa, transformamos os dados brutos da camada **Bronze** em uma tabela otimizada e confiável. O objetivo é garantir que os dados estejam limpos, tipados e prontos para o consumo analítico.

### 🛠️ Transformações Aplicadas:
* **🧹 Limpeza e Padronização**: Remoção de espaços em branco com `TRIM()` e normalização dos tipos de cervejaria para letras minúsculas com `LOWER()`.
* **📐 Tipagem Forte (Casting)**: Conversão de IDs para `STRING` e coordenadas geográficas para `DOUBLE` para garantir precisão nos cálculos.
* **🛡️ Qualidade de Dados**: Implementação de filtros para garantir que registros sem identificador único (`id`) não sigam no pipeline.
* **📍 Localização**: Uso de `COALESCE()` para garantir que a coluna de estado/província nunca seja nula, facilitando o agrupamento posterior.

In [0]:
USE CATALOG `bees-teste-jp`;
USE SCHEMA default;

In [0]:
CREATE OR REPLACE TEMPORARY VIEW vw_openbrewery_silver_prep AS
SELECT 
    CAST(id AS STRING) AS brewery_id,
    TRIM(name) AS name,
    LOWER(brewery_type) AS brewery_type,
    city,
    COALESCE(state_province, 'Unknown') AS state_province, 
    country,
    CAST(longitude AS DOUBLE) AS longitude,
    CAST(latitude AS DOUBLE) AS latitude,
    from_utc_timestamp(current_timestamp(), 'America/Sao_Paulo') AS updated_at
FROM bronze_openbrewery
WHERE id IS NOT NULL;

# 📂 Persistência e Resiliência do Pipeline

Para atender aos requisitos do Arquiteto, a camada Silver utiliza o formato **Delta Lake** (colunar) e uma estratégia de carga inteligente.

### 💎 Diferenciais da Implementação:
* **📌 Particionamento**: A tabela é fisicamente particionada por `state_province`, otimizando a performance de consultas por localização.
* **🔄 Idempotência com MERGE**: Utilizamos a operação de `MERGE` em vez de um simples `INSERT`. Isso garante que o pipeline possa ser reexecutado sem duplicar dados em caso de falhas.
* **🔍 Deduplicação**: Através da função `ROW_NUMBER()`, garantimos que apenas a versão mais recente de cada registro seja persistida na tabela final.
* **⏱️ Auditoria**: Inclusão de um timestamp de atualização (`updated_at`) para monitorar o frescor dos dados.

In [0]:
CREATE TABLE IF NOT EXISTS silver_openbrewery (
    brewery_id STRING,
    name STRING,
    brewery_type STRING,
    city STRING,
    state_province STRING,
    country STRING,
    longitude DOUBLE,
    latitude DOUBLE,
    updated_at TIMESTAMP
)
USING DELTA
PARTITIONED BY (state_province);

MERGE INTO silver_openbrewery AS target
USING (
    SELECT * FROM (
        SELECT *, ROW_NUMBER() OVER(PARTITION BY brewery_id ORDER BY updated_at DESC) as rn
        FROM vw_openbrewery_silver_prep
    ) WHERE rn = 1
) AS source
ON target.brewery_id = source.brewery_id

WHEN MATCHED THEN
  UPDATE SET 
    target.name = source.name,
    target.brewery_type = source.brewery_type,
    target.city = source.city,
    target.state_province = source.state_province,
    target.country = source.country,
    target.longitude = source.longitude,
    target.latitude = source.latitude,
    target.updated_at = source.updated_at

WHEN NOT MATCHED THEN
  INSERT (
    brewery_id, 
    name, 
    brewery_type, 
    city, 
    state_province, 
    country, 
    longitude, 
    latitude, 
    updated_at
  )
  VALUES (
    source.brewery_id, 
    source.name, 
    source.brewery_type, 
    source.city, 
    source.state_province, 
    source.country, 
    source.longitude, 
    source.latitude, 
    source.updated_at
  );